In [ ]:
import suspect
import pandas as pd
import numpy as np
import glob
import os
import json
import shutil

In [ ]:
#basis = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/LCModel_basis/7T_SE_TE9_NoMM.basis'
#basis = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/NewBasis/7T_SPECIAL_real_TE9.basis'
basis = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/NewBasis/7T_SPECIAL_real_TE9_NoMM.basis'

In [ ]:
fullspecs = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/suspect_preprocessing/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special*hlsvd.raw')
fullspecs

In [ ]:
water = [i.replace('spec-lr-specialavg_ecc_hlsvd.raw', 'spec-lr-special_water_wOVSavg.raw') for i in fullspecs]

In [ ]:
water

In [ ]:
[os.path.isfile(i) for i in water]

In [ ]:
te=9
tr=6500
R_GM = np.exp(-te/50)*(1-np.exp(-tr/2050))
R_WM = np.exp(-te/55)*(1-np.exp(-tr/1210))
R_CSF = np.exp(-te/1050)*(1-np.exp(-tr/4430))
# Water sds:
frac_jsons = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/tissue_frac/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special_segmentation.json')
fracs = dict()
for i in frac_jsons:
    sid = i.split('/')[-3]
    loc = i.split('_')[-3].split('-')[1]
    with open(i,'r') as fin:
        frac = json.load(fin)
    if not sid in fracs.keys():
        fracs[sid]=dict()
    fracs[sid][loc] = frac
for sid in fracs.keys():
    for roi in fracs[sid].keys():
        #fracs[sid][roi]['H2O'] = 55126 * (fracs[sid][roi]['WM'] * 0.65 + fracs[sid][roi]['GM'] * 0.78 + fracs[sid][roi]['CSF'] * 0.97)
        fracs[sid][roi]['H2O'] = 55126 * (fracs[sid][roi]['WM'] * 0.65 * R_WM + fracs[sid][roi]['GM'] * 0.78 *R_GM + fracs[sid][roi]['CSF'] * 0.97 * R_CSF)
        fracs[sid][roi]['atth2o'] = 1 # np.exp(-9/55)*fracs[sid][roi]['WM'] + np.exp(-9/50) *fracs[sid][roi]['GM'] + np.exp(-9/1050)*fracs[sid][roi]['CSF']

In [ ]:
# Copy file to derivatives Directory

In [ ]:
for i in fullspecs:
    outname = i.replace('derivatives/suspect_preprocessing', 'derivatives/PRaMM1_23')
    os.makedirs(os.path.dirname(outname), exist_ok=True)
    shutil.copyfile(i, outname)

In [ ]:
for i in water:
    outname = i.replace('derivatives/suspect_preprocessing', 'derivatives/PRaMM1_23')
    os.makedirs(os.path.dirname(outname), exist_ok=True)
    shutil.copyfile(i, outname)

In [ ]:
#Dev 20.05-99
pramm_control = '''
$LCMODL
title='PRaMM_{sub}_{roi}'
atth2o= {atth2o}
wconc= {h2oconc}
key = 210387309
deltat= 1.250e-04
doecc= F
dows= T
hzpppm= 2.9721e+02
neach= 999
nunfil= 4096
SDDEGZ = 1
SDDEGP = 0
shifmn = -0.05
shifmx = 0.05
FIXSHF = T
ppmend= 0.2
ppmst= 4.2
dkntmn= 99
echot= 9
nsimul= 15
chsimu(15) = 'M4.04 @ 4.05 +- .05 FWHM=0.001 < 0.08 +- .005 AMP=1'
chsimu(14) = 'M3.90 @ 3.92 +- .02 FWHM=0.08 < 0.10 +- .005 AMP=1'
chsimu(13) = 'M3.79 @ 3.79 +- .01 FWHM=0.02 < 0.08 +- .001 AMP=1'
chsimu(12) = 'M3.71 @ 3.71 +- .02 FWHM=0.04 < 0.10 +- .005 AMP=1'
chsimu(11) = 'M3.59 @ 3.59 +- .02 FWHM=0.04 < 0.07 +- .005 AMP=1'
chsimu(10) = 'M3.21 @ 3.21 +- .005 FWHM= 0.14 < 0.16 +- .01 AMP=1'
chsimu(9) = 'M3.00 @ 3.00 +- .005 FWHM= 0.10 < 0.12 +- .01 AMP=1.'
chsimu(8) = 'M2.74 @ 2.75 +- .01 FWHM= 0.10 < 0.13 +- .005 AMP=1.'
chsimu(7) = 'M2.54 @ 2.59 +- .01 FWHM= 0.11 < 0.14 +- .005 AMP=1.'
chsimu(6) = 'M2.27 @ 2.27  +- .005 FWHM= 0.09 < 0.15 +- .01 AMP=1.'
chsimu(5) = 'M2.05 @ 2.05 +- .005 FWHM= 0.15 < 0.28 +- .02 AMP=1.'
chsimu(4) = 'M1.70 @ 1.70 +- .01 FWHM= 0.17 < 0.19 +- .01 AMP= 1.'
chsimu(3) = 'M1.43 @ 1.43 +- .01 FWHM= 0.08 < 0.11 +- .01 AMP= 1.'
chsimu(2) = 'M1.22 @ 1.22 +- .01 FWHM= 0.07 < 0.09 +- .005 AMP= 1.'
chsimu(1) = 'M0.94 @ 0.94 +- .005 FWHM= 0.08 < 0.11 +- .01 AMP= 1.'
nratio = 14
chrato(1) = 'M1.70/M0.94 = 2.02 +- 0.04'
chrato(2) = 'M2.05/M1.43 = 2.47 +- 0.24'
chrato(3) = 'M2.74/M2.54 = 0.81 +- 0.08'
chrato(4) = 'M3.00/M0.94 = 2.13 +- 0.21'
chrato(5) = 'M3.00/M1.70 = 0.97 +- 0.09'
chrato(6) = 'M3.00/M2.74 = 3.53 +- 0.35'
chrato(7) = 'M3.21/M1.70 = 0.83 +- 0.08'
chrato(8) = 'M3.21/M2.74 = 3.78 +- 0.38'
chrato(9) = 'M3.21/M3.00 = 0.96 +- 0.10'
chrato(10) = 'M3.59/M3.21 = 0.08 +- 0.008'
chrato(11) = 'M3.71/M3.59 = 1.34 +- 0.13'
chrato(12) = 'M3.79/M3.59 = 1.05 +- 0.10'
chrato(13) = 'M3.79/M3.71 = 0.61 +- 0.06'
chrato(14) = 'M3.90/M3.79 = 1.58 +- 0.16'
nomit= 12
chomit= '-CrCH2' 'Gua' 'Ser' 'Lip13a' 'Lip13b' 'Lip09' 'MM09' 'Lip20' 'MM20' 'MM12' 'MM14' 'MM17'
chcomb= 'GPC+PCho'
FILBAS='{basis}'
FILRAW='{filename}.raw'
LTABLE=7, FILTAB='{filename}/{filename}.table'
FILPS='{filename}.ps'
LCSV=11, FILCSV='{filename}/{filename}.csv'
LCOORD=9, FILCOO='{filename}/{filename}.coord'
LPRINT=6, FILPRI='{filename}/{filename}.print'
LCORAW=10, FILCOR='{filename}/{filename}.coraw'
filh2o='{water}'
$END'''

In [ ]:
os.makedirs

In [ ]:
for n,i in enumerate(fullspecs):
    control_name = i.replace('derivatives/suspect_preprocessing', 'derivatives/PRaMM1_23').replace('.raw', '_PRaMM1.control')
    print(control_name)
    file_name = os.path.basename(i).rstrip('.raw')
    sub=file_name.split('_')[0]
    roi=file_name.split('_')[2]
    #print(file_name)
    print(control_name.replace('_PRaMM1.control',''))
    with open(control_name, 'w') as ctrl:
        ctrl.write(pramm_control.format(basis=basis, 
                                        filename=file_name,
                                        water=os.path.basename(water[n]),
                                        sub=sub,
                                        roi=roi,
                                       atth2o=fracs[sub][roi.split('-')[-1]]['atth2o'],
                                       h2oconc=fracs[sub][roi.split('-')[-1]]['H2O']))

In [ ]:
import subprocess

In [ ]:
import sys
sys.path.append('/home/orco/data/MacroMols/MRS/MRSpype')
from mrspype.utils import utils

In [ ]:
spec_ctrls = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/PRaMM1_23/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special*hlsvd*_PRaMM1.control'

In [ ]:
spec_ctrls = glob.glob(spec_ctrls)

In [ ]:
spec_ctrls

In [ ]:
len(spec_ctrls)

In [ ]:
curr_dir = os.getcwd()
curr_dir

In [ ]:
from joblib import Parallel, delayed

def lcm(i):
    print(i)
    dirname = os.path.dirname(i)
    print(dirname)
    os.chdir(dirname)
    #print(os.getcwd())
    os.makedirs(i.replace('_PRaMM1.control',''), exist_ok=True)
    print(i.replace('_PRaMM1.control',''))
    proc = subprocess.run('/home/orco/.lcmodel/bin/lcmodel < {filename}'.format(filename=os.path.basename(i)), shell=True, check=True, capture_output=True)
    print(proc)
    


Parallel(n_jobs=32)(delayed(lcm)(i) for i in spec_ctrls)

In [ ]:
# for i in spec_ctrls:
#     print(i)
#     dirname = os.path.dirname(i)
#     print(dirname)
#     os.chdir(dirname)
#     #print(os.getcwd())
#     os.makedirs(i.replace('_PRaMM.control',''), exist_ok=True)
#     print(i.replace('_PRaMM.control',''))
#     proc = subprocess.run('/home/orco/.lcmodel/bin/lcmodel < {filename}'.format(filename=os.path.basename(i)), shell=True, check=True, capture_output=True)
#     print(proc)
    


In [ ]:
import importlib
importlib.reload(utils)

In [ ]:
help(utils)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fullspecs

In [ ]:
fullspecs = [i.replace('derivatives/LC_Model', 'derivatives/PRaMM1_23') for i in fullspecs]

In [ ]:
coords_pramm = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/PRaMM1_23/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special*hlsvd/*.coord')

In [ ]:
coords_pramm

In [ ]:
len(coords_pramm)

In [ ]:
coords_scm = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/SCM/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special*/sub-*_nuc-1H_loc-*_spec-lr-special*hlsvd*.coord')

In [ ]:
coords_scm

In [ ]:
len(coords_scm)

In [ ]:
#coords_sub = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/IMNS/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special*/sub-*_nuc-1H_loc-*_spec-lr-special*.coord')
coords_sub = [i.replace('SCM','IMNS').replace('lr-specialavg_ecc_hlsvd','lr-special_subtracted') for i in coords_scm]

In [ ]:
len(coords_sub)

In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import suspect
import parsley
import seaborn as sns
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [10, 7]

In [ ]:
basis_grammar = r"""
namelist = '$' name:n pairs:p ws -> (n, dict(p))
pairs = (pair:first (pair)*:rest -> [first] + rest) | -> []
pair = ws name:k ws '=' ws valuelist:v ws ','? -> (k.upper(), v)
valuelist = ws (stringlist|numberlist|truth|falsehood):v -> v
truth = 'T' -> True
falsehood = 'F' -> False
stringlist = (string:first (ws string)+:rest -> [first] + rest) | string
string = '\'' (~'\'' anything)*:c '\'' -> ''.join(c).strip()
numberlist = (number:first (ws number)+:rest -> [first] + rest) | number
number = ('-' | -> ''):sign (digits:ds (floatPart(sign ds) | -> int(sign + ds)))
digits = <digit+>
name = <letter (letter | digit | '_')*>
floatPart :sign :ds = <('.' digits? exponent?) | exponent>:tail -> float(sign + ds + tail)
exponent = ('e' | 'E') ('+' | '-')? digits
"""

parser = parsley.makeGrammar(basis_grammar, {})


In [ ]:
def read_svs_raw(raw_path):
    with open(raw_path,'r') as fin:
        data = fin.read().lstrip()
        raw = dict()
        while data.startswith('$'):
            namelist_data, _, data = data.partition('$END')
            namelist = parser(namelist_data).namelist()
            raw[namelist[0]] = namelist[1]
            data = data.lstrip()
            if namelist[0].upper() == 'NMID':
                #then is the last section and the time domain data begin
                data = [i.lstrip().split() for i in data.split('\n') if i]
                iterable = (complex(float(i[0]),float(i[1])) for i in data)
                raw['time_domain'] = np.fromiter(iterable, 'complex64', len(data))
                #data is not anymore string
                break
    raw['freq_domain'] = np.fft.fftshift(np.fft.fft(raw['time_domain'], axis=-1), axes=-1)
    
    return raw

def scale_dbn(data, baseline, fit):
    from sklearn.preprocessing import MinMaxScaler
    noise = np.array(data) - np.array(fit)
    scaler = MinMaxScaler()
    scaler.fit(np.array(data).reshape(-1,1))
    new_data = scaler.transform(np.array(data).reshape(-1,1))[:,0]
    new_baseline = scaler.transform(np.array(baseline).reshape(-1,1))[:,0]
    new_noise = scaler.transform(noise.reshape(-1,1))[:,0]
    return new_data, new_baseline, new_noise
    
def scale_01(data):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(np.array(data).reshape(-1,1))
    return scaler.transform(np.array(data).reshape(-1,1))[:,0]

In [ ]:
def scaled_raw_pure_noise(raw_path):
    # Read RAW
    # Scale the real part of frequency domain to 0 to 1
    # Return the variance of pure noise, points between +- -1 to 0 ppm (points 2750 to 3250)
    return scale_01(read_svs_raw(raw_path)['freq_domain'].real)[2750:3500]
    
def get_fit_rest(coord):
    c = suspect.io.lcmodel.read_coord(coord)
    _, bl, fit_noise = scale_dbn(c['data'], c['baseline'], c['fit'])
    return bl + fit_noise

def get_fit_noise(coord):
    c = suspect.io.lcmodel.read_coord(coord)
    _, _, fit_noise = scale_dbn(c['data'], c['baseline'], c['fit'])
    return fit_noise


def get_fqn(fit_rest, pure_noise):
    return np.var(fit_rest)/np.var(pure_noise)

In [ ]:
full_raws = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/SCM/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-specialavg_ecc_hlsvd.raw'
full_raws = glob.glob(full_raws)
#full_raws

In [ ]:
var_scaled_raw_pure_noise = [np.var(scaled_raw_pure_noise(i)) for i in full_raws]

In [ ]:
def get_fit_rest(coord):
    c = suspect.io.lcmodel.read_coord(coord)
    _, bl, fit_noise = scale_dbn(c['data'][350:], c['baseline'][350:], c['fit'][350:])
    return  bl #+fit_noise #bl + 

In [ ]:
#fit_rest_1 = [np.var(get_fit_rest(i)) for i in coords_pramm]
#fit_rest_125 = [np.var(get_fit_rest(i)) for i in coords_pramm025]
#fit_rest_2 = [np.var(get_fit_rest(i)) for i in coords_scm]
#fit_rest_3 = [np.var(get_fit_rest(i)) for i in coords_sub]

In [ ]:
len(coords_scm)

In [ ]:
fit_rest_3 = list()
for i in coords_sub:
    #print(i)
    try:
        r = np.var(get_fit_rest(i))
    except:
        print(i)
        r = np.nan
    fit_rest_3.append(r)

In [ ]:
fit_rest_3

In [ ]:
fit_rest_1 = list()
for i in coords_pramm:
    try:
        r = np.var(get_fit_rest(i))
    except:
        r = np.nan
        print(i)
    fit_rest_1.append(r)

In [ ]:
fit_rest_2 = list()
for i in coords_scm:
    try:
        r = np.var(get_fit_rest(i))
    except Exception as e:
        r = np.nan
        print(i, e)
    fit_rest_2.append(r)

In [ ]:
# fit_rest_125 = list()
# for i in coords_pramm025:
#     try:
#         r = np.var(get_fit_rest(i))
#     except:
#         r = np.nan
#         print(i)
#     fit_rest_125.append(r)

In [ ]:
np.array(fit_rest_2)

In [ ]:
fqn_1 = np.array(fit_rest_1) / np.array(var_scaled_raw_pure_noise)
fqn_2 = np.array(fit_rest_2) / np.array(var_scaled_raw_pure_noise)
fqn_3 = np.array(fit_rest_3) / np.array(var_scaled_raw_pure_noise)

In [ ]:
fqn_df = pd.DataFrame.from_dict({'pramm':fqn_1,
                                 'scm': fqn_2,
                                'imns': fqn_3})

In [ ]:
sns.boxplot(y = 'value',x = 'variable',  data = pd.melt(fqn_df), palette = 'coolwarm').set_title('FQN baseline + noise')

In [ ]:
def get_fit_restn(coord):
    c = suspect.io.lcmodel.read_coord(coord)
    _, bl, fit_noise = scale_dbn(c['data'][350:], c['baseline'][350:], c['fit'][350:])
    return  fit_noise #bl + 

In [ ]:
# fit_rest_1n = [np.var(get_fit_restn(i)) for i in coords_pramm]
# fit_rest_125n = [np.var(get_fit_restn(i)) for i in coords_pramm025]
# fit_rest_2n = [np.var(get_fit_restn(i)) for i in coords_scm]
# fit_rest_3n = [np.var(get_fit_restn(i)) for i in coords_sub]

In [ ]:
fit_rest_1n = list()
for i in coords_pramm:
    try:
        r = np.var(get_fit_restn(i))
    except:
        r = np.nan
        print(i)
    fit_rest_1n.append(r)

In [ ]:
# fit_rest_125n = list()
# for i in coords_pramm025:
#     try:
#         r = np.var(get_fit_restn(i))
#     except:
#         r = np.nan
#         print(i)
#     fit_rest_125n.append(r)

In [ ]:
fit_rest_2n = list()
for i in coords_scm:
    try:
        r = np.var(get_fit_restn(i))
    except:
        r = np.nan
        print(i)
    fit_rest_2n.append(r)

In [ ]:
fit_rest_3n = list()
for i in coords_sub:
    try:
        r = np.var(get_fit_restn(i))
    except:
        r = np.nan
        print(i)
    fit_rest_3n.append(r)

In [ ]:
fqn_1n = np.array(fit_rest_1n) / np.array(var_scaled_raw_pure_noise)
fqn_2n = np.array(fit_rest_2n) / np.array(var_scaled_raw_pure_noise)
fqn_3n = np.array(fit_rest_3n) / np.array(var_scaled_raw_pure_noise)

In [ ]:
fqn_dfn = pd.DataFrame.from_dict({'pramm':fqn_1n,
                                 'scm': fqn_2n,
                                 'imns': fqn_3n})
sns.boxplot(y = 'value',x = 'variable',  data = pd.melt(fqn_dfn), palette = 'coolwarm').set_title('FQN_noise_only')

In [ ]:
coords_pramm[0]

In [ ]:
sids = [i.split('/')[-4] for i in coords_pramm]
sids

In [ ]:
locs = [i.split('_')[-4].split('-')[1] for i in coords_pramm]
locs

In [ ]:
stats.ttest_rel(fqn_1, fqn_2)

In [ ]:
stats.ttest_rel(fqn_1, fqn_2)

In [ ]:
fqn_df
fqn_df['sids'] = sids

In [ ]:
fqn_df

In [ ]:
#fqn_df['sub'] = list(fqn_df['sub'].iloc[1:3]) + [np.nan] + list(fqn_df['sub'].iloc[3:])


In [ ]:
fqn_df['metric'] = 'FQN_noise_and_BL'
fqn_df['location'] = locs
fqn_df

In [ ]:
#fqn_dfn['sub'] = list(fqn_dfn['sub'].iloc[1:3]) + [np.nan] + list(fqn_dfn['sub'].iloc[3:])
fqn_dfn['metric'] = 'FQN_noise'
fqn_dfn['location'] = locs
fqn_dfn

In [ ]:
fqn_dfn
fqn_dfn['sids'] = sids

In [ ]:
df = pd.concat([fqn_dfn, fqn_df])
df

In [ ]:
len(sids)

In [ ]:
def fwhm(i):
    try:
        return suspect.io.lcmodel.read_coord(i)['misc_output']['fwhm']
    except:
        return np.nan
    
fwhm_df = pd.DataFrame.from_dict({'pramm':[fwhm(i) for i in coords_pramm],
                                 'scm': [fwhm(i) for i in coords_scm],
                                 'imns': [fwhm(i) for i in coords_sub]})
sns.boxplot(y = 'value',x = 'variable',  data = pd.melt(fwhm_df), palette = 'coolwarm').set_title('fwhm')

In [ ]:
def snr(i):
    try:
        return suspect.io.lcmodel.read_coord(i)['misc_output']['snr']
    except:
        return np.nan
snr_df = pd.DataFrame.from_dict({'pramm':[snr(i) for i in coords_pramm],
                                 'scm': [snr(i) for i in coords_scm],
                                 'imns': [snr(i) for i in coords_sub]})
sns.boxplot(y = 'value',x = 'variable',  data = pd.melt(snr_df), palette = 'coolwarm').set_title('snr')


In [ ]:
snr_df

In [ ]:
#snr_df['sub'] = list(snr_df['sub'].iloc[1:3]) + [np.nan] + list(snr_df['sub'].iloc[3:])
snr_df['metric'] = 'SNR'
snr_df['sids'] = sids
snr_df['location'] = locs
snr_df

In [ ]:
df = pd.concat([df, snr_df])
df

In [ ]:
df2 = df.copy()

In [ ]:
#df = df2.copy()

In [ ]:
coords_pramm[1]

In [ ]:
suspect.io.lcmodel.read_coord(coords_pramm[1])['metabolite_fits']

In [ ]:
met_mm = list(suspect.io.lcmodel.read_coord(coords_pramm[1])['metabolite_fits'].keys())
met_mm = [i.lstrip('+').lstrip('-') for i in met_mm]
met_mm

In [ ]:
df = df2.copy()
#df

In [ ]:
met_mm

In [ ]:
for j in met_mm:
    p = []
    p025 = []
    s = []
    im = []
    try:
        if j[0] == 'M':
            p=list()
            for i in coords_pramm:
                try:
                    m = suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['concentration']
                except:
                    try:
                        m=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['+{}'.format(j)]['concentration']
                    except:
                        try:
                            m=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['-{}'.format(j)]['concentration']
                        except:
                            m=np.nan
                p.append(m)
        else:
            for i in coords_pramm:
                try:
                    m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['concentration']
                except:
                    m=np.nan
                p.append(m)
    except Exception as e:
        print(e)
        p = [np.nan] * len(sids)
    try:
        if j[0] == 'M':
            p025=list()
            for i in coords_pramm025:
                try:
                    m025 = suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['concentration']
                except:
                    try:
                        m025=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['+{}'.format(j)]['concentration']
                    except:
                        try:
                            m025=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['-{}'.format(j)]['concentration']
                        except:
                            m025=np.nan
                p025.append(m025)
        else:
            for i in coords_pramm025:
                try:
                    m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['concentration']
                except:
                    m=np.nan
                p025.append(m)
    except:
        p025 = [np.nan] * len(sids)
    try:
        for i in coords_scm:
            try:
                m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['concentration']
            except:
                m=np.nan
            s.append(m)
    except:
        s = [np.nan] * len(sids)
    try:
        for i in coords_sub:
            try:
                m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['concentration']
            except:
                m=np.nan
            im.append(m)
    except:
        im = [np.nan] * len(sids)
    tmp_df = pd.DataFrame.from_dict({'pramm':p,
                                     'scm': s,
                                     'imns': im})
    #tmp_df['sub'] = list(tmp_df['sub'].iloc[1:3]) + [np.nan] + list(tmp_df['sub'].iloc[3:])
    tmp_df['metric'] = 'conc_{}'.format(j)
    tmp_df['sids'] = sids
    tmp_df['location'] = locs
    #print(tmp_df)

    df = pd.concat([df, tmp_df])

In [ ]:
df

In [ ]:
df[(df.metric == 'conc_Cr')]

In [ ]:
for j in met_mm:
    p = []
    p025 = []
    s = []
    im = []
    try:
        if j[0] == 'M':
            p=list()
            for i in coords_pramm:
                try:
                    m = suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['sd']
                except:
                    try:
                        m=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['+{}'.format(j)]['sd']
                    except:
                        try:
                            m=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['-{}'.format(j)]['sd']
                        except:
                            m=np.nan
                p.append(m)
        else:
            for i in coords_pramm:
                try:
                    m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['sd']
                except:
                    m=np.nan
                p.append(m)
    except Exception as e:
        print(e)
        p = [np.nan] * len(sids)
    try:
        if j[0] == 'M':
            p025=list()
            for i in coords_pramm025:
                try:
                    m025 = suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['sd']
                except:
                    try:
                        m025=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['+{}'.format(j)]['sd']
                    except:
                        try:
                            m025=suspect.io.lcmodel.read_coord(i)['metabolite_fits']['-{}'.format(j)]['sd']
                        except:
                            m025=np.nan
                p025.append(m025)
        else:
            for i in coords_pramm025:
                try:
                    m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['sd']
                except:
                    m=np.nan
                p025.append(m)
    except:
        p025 = [np.nan] * len(sids)
    try:
        for i in coords_scm:
            try:
                m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['sd']
            except:
                m=np.nan
            s.append(m)
    except:
        s = [np.nan] * len(sids)
    try:
        for i in coords_sub:
            try:
                m=suspect.io.lcmodel.read_coord(i)['metabolite_fits'][j]['sd']
            except:
                m=np.nan
            im.append(m)
    except:
        im = [np.nan] * len(sids)
    tmp_df = pd.DataFrame.from_dict({'pramm':p,
                                     'scm': s,
                                     'imns': im})
    #tmp_df['sub'] = list(tmp_df['sub'].iloc[1:3]) + [np.nan] + list(tmp_df['sub'].iloc[3:])
    tmp_df['metric'] = 'CRLB_{}'.format(j)
    tmp_df['sids'] = sids
    tmp_df['location'] = locs
    #print(tmp_df)

    df = pd.concat([df, tmp_df])

In [ ]:
df

In [ ]:
df.metric.unique()

In [ ]:
df.to_csv('./23_all_metrics_water.csv')

In [ ]:
a="""relax.Asc   = [1530, 1484, 127, 128]; % This is the average from tNAA, tCr, tCho, Glx, and mI
        relax.Asp   = [1530, 1484, 127, 128]; % This is the average from tNAA, tCr, tCho, Glx, and mI
        relax.Cr    = [1740, 1780, 107, 107]; % Taken from tCr
        relax.GABA  = [1334, 1334, 87, 87]; % Andreychenko et al. (2012) 10.1002/nbm.2997
        relax.Glc   = [1530, 1484, 127, 128]; % This is the average from tNAA, tCr, tCho, Glx, and mI
        relax.Gln   = [1640, 1740, 107, 107]; %T1 from Mlynarik et al. (2012) 10.1002/mrm.24352 % T2 as Gln
        relax.Glu   = [1610, 1750, 107, 117]; % T1 from Mlynarik et al. (2012) 10.1002/mrm.24352, T2 from https://doi.org/10.1371/journal.pone.0215210
        relax.Gly   = [1530, 1484, 127, 128]; % This is the average from tNAA, tCr, tCho, Glx, and mI
        relax.GPC   = [1510, 1320,  153, 153]; % Taken from tCho
        relax.GSH   = [1140, 1060, 79, 79]; % Entire molecule; T1 from Mlynarik et al. (2012) 10.1002/mrm.24352
        relax.Lac   = [1530, 1484, 182, 182]; % The use of MEGA-sLASER with J-refocusing echo time extension to measure the proton T2 of lactate in healthy human brain at 7 T ISMRM
        relax.mI   = [1280, 1190, 111, 111]; %T1 from Mlynarik et al. (2012) 10.1002/mrm.24352
        relax.NAA   = [1780, 1830, 155, 155]; % This is the 2.008 ppm acetyl signal (naa in the paper); aspartyl is naa1: [1310 1310 110 110]; T1 from Mlynarik et al. (2012) 10.1002/mrm.24352
        relax.NAAG  = [1210, 940, 155, 155]; % This is the 2.042 ppm acetyl signal (naag in the paper); aspartyl is naag1: [1310 1310 (108+87)/2 180]; % glutamate is NAAG2: [1310 1310 (110+78)/2 157];
        relax.PCh   = [1510, 1320,  153, 153]; % Taken from tCho
        relax.PCr   = [1740, 1780, 107, 107]; % Taken from tCr
        relax.PE    = [1310, 1320, 182, 182]; %T1 from Mlynarik et al. (2012) 10.1002/mrm.24352; No reference values available. T2s copied from Lac (closest match at 3T)
        relax.sI   = [1310, 1230, 105, 105]; %T1 from Mlynarik et al. (2012) 10.1002/mrm.24352 T2 from https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.24352
        relax.Tau   = [2150, 2090, 97, 97]; %T1 from Mlynarik et al. (2012) 10.1002/mrm.24352 % T2 as Gln
        relax.tNAA  = [1495, 1385, 155, 155]; % Mean values from NAA + NAAG
        relax.tCr  = [1740, 1780, 107, 107]; % The singlet peak ar 3 ppm. 3.9 ppm peak values are [1240 1190 94 94] %T1 from Mlynarik et al. (2012)
        relax.tCho  = [1510, 1320,  153, 153]; % Entire molecule; T1 from Mlynarik et al. (2012) 10.1002/mrm.24352
        relax.Glx  = [1625, 1745, 107, 112]; % Mean values from Glu + Glx"""

In [ ]:
l = a.replace('relax.','').replace('= [',',').replace('];',',').split('\n')

In [ ]:
l = [i.split(',') for i in l]


In [ ]:
ll=[]
for i in l:
    ll.append([j.rstrip().lstrip() for j in i])

In [ ]:
relax_df = pd.DataFrame(ll)

In [ ]:
relax_df.columns = [['Metab', 'T1_GM', 'T1_WM', 'T2_GM', 'T2_WM', 'Desc','Desc','Desc','Desc','Desc',]]

In [ ]:
relax_df['T1_avg'] = (relax_df.T1_GM.values.reshape(23).astype(np.int64) + relax_df.T1_WM.values.reshape(23).astype(np.int64))/2
relax_df['T2_avg'] = (relax_df.T2_GM.values.reshape(23).astype(np.int64) + relax_df.T2_WM.values.reshape(23).astype(np.int64))/2
relax_df

In [ ]:
relax_df.T1_WM.values.reshape(23).astype(np.int64)

In [ ]:
relax_df.to_csv('relax_df.csv')

In [ ]:
relax_dict = pd.read_csv('relax_df.csv').set_index('Metab').to_dict(orient='index')

In [ ]:
#relax_dict

In [ ]:
df

In [ ]:
df['CSF'] = [fracs[i][j]['CSF'] for i,j in zip(df.sids,df.location)]
df['GM'] = [fracs[i][j]['GM'] for i,j in zip(df.sids,df.location)]
df['WM'] = [fracs[i][j]['WM'] for i,j in zip(df.sids,df.location)]

In [ ]:
df['TE']=9
df['TR']=6500

In [ ]:
fracs[list(df.sids)[0]][list(df.location)[1]]

In [ ]:
df.metric.unique()

In [ ]:
alt_name = ['Asc', 'Asp', 'Cr', 'GABA', 'Glc', 'Gln', 'Glu', 'Gly', 'GPC', 'GSH', 'Lac', 'Ins', 'NAA', 'NAAG', 'PCho', 'PCr', 'PE', 'Scyllo', 'Tau', 'TNAA', 'TCr', 'tCho', 'Glx']

In [ ]:
relax_dict

In [ ]:
relax_dict['TNAA']=relax_dict['tNAA']
relax_dict['Ins']=relax_dict['mI']
relax_dict['Scyllo']=relax_dict['sI']
relax_dict['GPC+PCho']=relax_dict['tCho']
relax_dict['NAA+NAAG']=relax_dict['tNAA']
relax_dict['TCr']=relax_dict['tCr']
relax_dict['PCho']=relax_dict['PCh']

In [ ]:
relax_df = pd.DataFrame.from_dict(relax_dict, orient='index').drop(columns=['Unnamed: 0'])
relax_df

In [ ]:
#relax_df['alt_name'] = alt_name

In [ ]:
relax_df.to_csv('relax_df.csv')

In [ ]:
df[(df.metric == 'conc_Asp')]

In [ ]:
relax_df

In [ ]:
def corr_csf(c, T1, T2, f_csf, TE, TR):
    return c / (np.exp(-TE/T2) * (1-np.exp(-TR/T1))*(1-f_csf))

In [ ]:
for i in [i for i in list(df.metric.unique()) if i.startswith('conc_')]:
    alt_i = i.replace('conc_','')
    print(i, alt_i)
    if alt_i in relax_dict.keys():
        print(alt_i)
        df_tmp = df[(df.metric == i)]
        t1=relax_dict[alt_i]['T1_avg']
        t2=relax_dict[alt_i]['T2_avg']
        te=9
        tr=6500
        pramm = df_tmp['pramm'] / (np.exp(-te/t2) * (1-np.exp(-tr/t1))*(1-df_tmp['CSF']))
        scm = df_tmp['scm'] / (np.exp(-te/t2) * (1-np.exp(-tr/t1))*(1-df_tmp['CSF']))
        imns = df_tmp['imns'] / (np.exp(-te/t2) * (1-np.exp(-tr/t1))*(1-df_tmp['CSF']))
        df_tmp = df_tmp.drop(columns = ['pramm', 'scm', 'imns', 'metric'])
        df_tmp['pramm'] = pramm
        df_tmp['scm'] = scm
        df_tmp['imns'] = imns
        df_tmp['metric'] = 'abs_{}'.format(i)
        df = pd.concat([df, df_tmp])

In [ ]:
relax_dict.keys()

In [ ]:
# for i in [i for i in list(df.metric.unique()) if i.startswith('conc_M')]:
#     print(i)
#     pramm = df_tmp['pramm'] / (1-df_tmp['CSF'])
#     pramm025 = df_tmp['pramm025'] / (1-df_tmp['CSF'])
#     scm = df_tmp['scm'] / (1-df_tmp['CSF'])
#     imns = df_tmp['imns'] / (1-df_tmp['CSF'])
#     df_tmp = df_tmp.drop(columns = ['pramm', 'pramm025', 'scm', 'imns', 'metric'])
#     df_tmp['pramm']a = pramm
#     df_tmp['pramm025'] = pramm025
#     df_tmp['scm'] = scm
#     df_tmp['imns'] = imns
#     df_tmp['metric'] = 'csfcorr_{}'.format(i)
#     df = pd.concat([df, df_tmp])

In [ ]:
df.metric.unique()

In [ ]:
df[(df.metric=='abs_conc_PCho')]

In [ ]:
df[(df.metric=='abs_conc_GPC')]

In [ ]:
df_no = df[(df.metric!='abs_conc_GPC+PCho')].copy()

In [ ]:
df_no

In [ ]:
df_tcho = df[(df.metric=='abs_conc_GPC+PCho')].copy()

In [ ]:
df_tcho.scm = df[(df.metric=='abs_conc_PCho')].scm + df[(df.metric=='abs_conc_GPC')].scm

In [ ]:
df_tcho.imns = df[(df.metric=='abs_conc_PCho')].imns + df[(df.metric=='abs_conc_GPC')].imns

In [ ]:
df_tcho

In [ ]:
df = pd.concat([df_no, df_tcho])

In [ ]:
df.to_csv('./23_all_metrics_water_abs.csv')

In [ ]:
df.head()

In [ ]:
#df